<a href="https://colab.research.google.com/github/barthas1985/Stock_indicators_Warsaw_Stock_Exchange/blob/main/Biznesrdar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import requests
#import urllib.request
from bs4 import BeautifulSoup
import pandas as pd
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.198 Safari/537.36'}
# List for data that will be result of Looping Through Tabels.
wiersze = []
# Function that will extract/ parsing data from tabels (placed on website) accoridng to ticker name (companies listed on Warsaw Stock Exchange) and page number. 
def get_many(ticker, page):
  # Source of Data.
  url = f'https://www.biznesradar.pl/notowania-historyczne/{ticker},{page}' 
  # sending a request and return a content of the page
  r = requests.get(url, headers = headers)
  # 
  soup = BeautifulSoup(r.text, 'html.parser')
  
  tabela = soup.find('table', attrs = {'class':'qTableFull'})
  ilosc_wierszy =tabela.findAll('tr')
  
  for wiersz in ilosc_wierszy:
    dane = wiersz.findAll('td')
    #print(dana.text)
    if len(dane) == 0: 
        continue

    data = dane[0].getText()
    otwarcie = dane[1].getText()
    max = dane[2].getText()
    min = dane[3].getText()
    zamkniecie = dane[4].getText()
    wolumen = dane[5].getText()
    obrot = dane[6].getText()  
    wiersze.append([data, otwarcie, max, min, zamkniecie, wolumen, obrot])
  
  return

ticker = input()
for x in range(1,4):
  get_many(ticker,x)



PZU
True
True
True


In [3]:
df = pd.DataFrame(wiersze, columns = ['Data','Otwarcie','Max','Min','Zamknięcie','Wolumen','Obrót'])

#df.columns
df['Data'] = pd.to_datetime(df['Data']).dt.date
df['Otwarcie'] = df['Otwarcie'].astype(float)
df['Max'] = df['Max'].astype(float)
df['Min'] = df['Min'].astype(float)
df['Zamknięcie'] =df['Zamknięcie'].astype(float)
df['Wolumen'] = df['Wolumen'].str.replace(' ','')
df['Wolumen'] = df['Wolumen'].astype(int)
df['Obrót'] = df['Obrót'].str.replace(' ','')
df['Obrót'] = df['Obrót'].astype(int)


print(df.dtypes)



Data           object
Otwarcie      float64
Max           float64
Min           float64
Zamknięcie    float64
Wolumen         int64
Obrót           int64
dtype: object


In [ ]:
df


,Data,Otwarcie,Max,Min,Zamknięcie,Wolumen,Obrót
0,2021-10-11,127.50,130.20,125.00,126.35,983035,124309240
1,2021-09-11,128.10,129.50,127.00,127.40,684417,87454192
2,2021-08-11,130.00,130.30,127.80,129.35,404724,52247664
3,2021-05-11,130.00,131.75,128.60,129.80,691218,89841520
4,2021-04-11,128.10,130.10,126.40,129.50,1737045,224288464
...,...,...,...,...,...,...,...
145,2021-04-16,76.00,76.64,75.34,76.24,719168,54733816
146,2021-04-15,75.28,76.00,74.52,76.00,494167,37296556
147,2021-04-14,74.52,75.26,74.50,75.20,511140,38327152
148,2021-04-13,74.68,74.68,73.36,74.48,297526,22039706


In [4]:
df.to_excel('lista.xlsx')

In [ ]:
df

,Data,Otwarcie,Max,Min,Zamknięcie,Wolumen,Obrót
0,2021-08-11,6.330,6.380,6.220,6.220,2413484,15136460
1,2021-05-11,6.348,6.348,6.230,6.264,1612817,10122599
2,2021-04-11,6.210,6.360,6.210,6.348,3854629,24408430
3,2021-03-11,6.250,6.342,6.126,6.156,2751260,17110366
4,2021-02-11,6.066,6.294,5.992,6.146,5288009,32517124
...,...,...,...,...,...,...,...
145,2021-04-14,6.650,6.748,6.576,6.748,5493552,36704512
146,2021-04-13,6.610,6.640,6.474,6.608,5650762,37014256
147,2021-12-04,6.430,6.700,6.390,6.618,5348129,35171240
148,2021-09-04,6.446,6.496,6.360,6.434,11466427,73733576


In [ ]:
df = df.set_index(pd.DatetimeIndex(df['Data'].values))
df

,Data,Otwarcie,Max,Min,Zamknięcie,Wolumen,Obrót
2021-08-11,2021-08-11,6.330,6.380,6.220,6.220,2413484,15136460
2021-05-11,2021-05-11,6.348,6.348,6.230,6.264,1612817,10122599
2021-04-11,2021-04-11,6.210,6.360,6.210,6.348,3854629,24408430
2021-03-11,2021-03-11,6.250,6.342,6.126,6.156,2751260,17110366
2021-02-11,2021-02-11,6.066,6.294,5.992,6.146,5288009,32517124
...,...,...,...,...,...,...,...
2021-04-14,2021-04-14,6.650,6.748,6.576,6.748,5493552,36704512
2021-04-13,2021-04-13,6.610,6.640,6.474,6.608,5650762,37014256
2021-12-04,2021-12-04,6.430,6.700,6.390,6.618,5348129,35171240
2021-09-04,2021-09-04,6.446,6.496,6.360,6.434,11466427,73733576


Create SMA and EMA ...( all below is a part that I am still learning:) ) 

In [ ]:
def SMA(data, period=30, column = 'Zamknięcie'):
  return data[column].rolling(window=period).mean()
def EMA(data, period = 20, column ='Zamknięcie'):
  return data[column].ewm(span = period, adjust = False).mean()



MACD Moving Average Donvergence/Divergence


In [ ]:
def MACD(data, period_long = 26, period_short = 12, period_signal = 9, column = 'Zamknięcie'):
  # Short Term Expotential moving aavergae 
  ShortEMA = EMA(data, period_short, column = column)
  # Long EMA
  LongEMA = EMA(data, period_long, column = column)
  #Calc MACD
  data['MACD'] = ShortEMA - LongEMA
  #signal line
  data['Signal_Line'] = EMA(data, period_signal, column = 'MACD')
  return data

  